In [126]:
%load_ext bigdata

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [127]:
%pig_start

In [128]:
%timeout 300

In [129]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rm output2/*
!hadoop fs -rmdir input output
!hadoop fs -rmdir input output2

rm: `input/*': No such file or directory
rm: `output/*': No such file or directory
rm: `output2/*': No such file or directory
rmdir: `output': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output2': No such file or directory


In [130]:
!hadoop fs -put *.tsv .
!hadoop fs -ls

put: `data.tsv': File exists
Found 2 items
-rw-r--r--   1 root supergroup        615 2020-01-19 13:26 data.csv
-rw-r--r--   1 root supergroup       1741 2020-01-19 15:51 data.tsv


In [131]:
!rm -rf input
!rm -rf output
!mkdir input
!mkdir output

In [132]:
%%writefile script.pig

-- crea la carpeta input in el HDFS
-- fs -mkdir input

-- copia los archivos del sistema local al HDFS
fs -put input/ .

-- carga de datos
--Punto 6
u = LOAD 'data.tsv'
    AS (f1:CHARARRAY, f2:CHARARRAY, f3:MAP[]);
    
columna = FOREACH u GENERATE $2;
dato1 = FOREACH columna GENERATE FLATTEN($0);
agrupar = GROUP dato1 BY $0;
contar = FOREACH agrupar GENERATE group, COUNT($1) as ld1:CHARARRAY;
contar1 = FOREACH contar GENERATE CONCAT(group, ',' , ld1);

DUMP contar1;

-- escribe el archivo de salida
STORE contar1 INTO 'output';

-- copia los archivos del HDFS al sistema local
fs -get output/ .

Overwriting script.pig


In [133]:
!pig -execute 'run script.pig'

2020-01-19 17:18:22,763 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2020-01-19 17:18:26,286 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-01-19 17:18:26,771 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2020-01-19 17:18:26,782 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapreduce.output.fileoutputformat.compress
2020-01-19 17:18:26,825 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
2020-01-19 17:18:26,847 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.submit.replication is deprecated. Instead, use mapreduce.client.submit.file.

In [134]:
!hadoop fs -ls output/*

-rw-r--r--   1 root supergroup          0 2020-01-19 17:19 output/_SUCCESS
-rw-r--r--   1 root supergroup         70 2020-01-19 17:19 output/part-r-00000


In [135]:
!hadoop fs -cat output/part-r-00000

aaa,13
bbb,16
ccc,23
ddd,23
eee,15
fff,20
ggg,13
hhh,16
iii,18
jjj,18


In [136]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rm output2/*
!hadoop fs -rmdir input output
!hadoop fs -rmdir input output2

rm: `input/*': No such file or directory
Deleted output/_SUCCESS
Deleted output/part-r-00000
rm: `output2/*': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output2': No such file or directory


In [137]:
!rm -rf input
!rm -rf output